<a href="https://colab.research.google.com/github/slc2345/Algoritmos_Optimizacion/blob/main/Trabajo_SergioLidonFINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
### **Nombre y Apellidos: SERGIO LIDÓN CALVO** <br>



Url: https://github.com/slc2345/Algoritmos_Optimizacion.git<br>
Google Colab:https://colab.research.google.com/drive/1fOiX0gu0L_0moiT1lrorbyu3EYDhuOPW?usp=sharing <br>
Problema: Sesiones de doblaje <br>

Descripción del problema:

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben
coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible.







                                        

#Modelo
**- ¿Como represento el espacio de soluciones?**

El espacio de soluciones lo represento como una lista formada por sublistas que contienen las tomas que se grabarán en cada día: (lista[1]-> Lista que contiene el conjunto de tomas en el DÍA 1).

**- ¿Cual es la función objetivo?**

La función objetivo de este problema es minimizar el costo de la grabación total, minimizando la cantidad de veces que va un actor al set de rodaje. Esto se realiza agrupando las tomas asegurandonos de maximizar las coincidencias entre actores en las tomas, para minimizar el coste.


**- ¿Como implemento las restricciones?**

La restricción prinicpal de este problema es que en un día de grabación, se puede grabar máximo 6 tomas. En el método de búsqueda tabú, la restricción se realiza cuando se va a crear la planificación aleatoria, empleando subconjuntos de 6 tomas.

#Análisis
**- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones**

La complejidad de este problema es elevada, debido a que es un problema combinatorio que presenta un significativo espacio de soluciones. Al ser un problema de tipo binario (estar una toma o no en un día) de alto nivel de opciones, el orden de complejidad es de tipo exponencial: O(2^n), siendo n el número de tomas, en este caso, 30.

Respecto a contabilizar el espacio de soluciones, es un problema de combinaciones, ya que no importa el orden ni se permite repetir tomas, pero existe la restricción de 6 tomas MÁXIMO por día. El cálculo del espacio de soluciones se podría calcular de la siguiente forma: Cn,k = n! / (k! * (n-1)!), siendo n = 30 tomas y k el número de días, pero claro, hay que tener en cuenta la restricción de las tomas máximas por día y la coincidencia entre actores en las tomas de un día, por tanto, de esta forma no se podría calcular, se debe realizar métodos de optimización para calcularlo al ser un problema más complejo.

#Diseño
**- ¿Que técnica utilizo? ¿Por qué?**

La técnica empleada en este caso es de tipo **Heurística**, en concreto, **Búsqueda Tabú**.

En este caso se  emplea un algoritmo heurístico debido a que es un problema combinatorio con una gran cantidad de posibilidades al ser 30 tomas y 10 actores y este tipo de algoritmos nos sirve para obtener una solución óptima(que no exacta) en un tiempo razonable.

Existen métodos exactos, como Fuerza Bruta, los cuales realizan la búsqueda de la solución óptima en todo el espacio de soluciones, obteniendo el mejor de todos, pero el problema es que existe un gran gasto computacional al haber muchas combinaciones que explorar. Por tanto, el uso en este caso de un algoritmo de tipo heurístico aporta una gran ventaja, ya que, busca soluciones de buena calidad en un menor tiempo, con un menor coste computacional.

La búsqueda Tabú es una gran opción, debido a que es una técnica que, empleando la búsqueda local, genera nuevas soluciones vecinas a partir de una solución inicial. A partir de una lista tabú, en la cual se almacenan soluciones que ya se han visto, evitando su utilización en las siguientes iteraciones y de este modo, escapar de óptimos locales. La mejor solución hasta el momento, será la final.

In [1]:
# IMPORTS
import numpy as np
import random
import copy

In [2]:
# DATOS DEL PROBLEMA
max_tomas =  6 # Máximo de tomas por días

actores_tomas = np.array([[1,1,1,1,1,0,0,0,0,0],[0,0,1,1,1,0,0,0,0,0],[0,1,0,0,1,0,1,0,0,0],[1,1,0,0,0,0,1,1,0,0],[0,1,0,1,0,0,0,1,0,0],[1,1,0,1,1,0,0,0,0,0],[1,1,0,1,1,0,0,0,0,0],[1,1,0,0,0,1,0,0,0,0],[1,1,0,1,0,0,0,0,0,0],
[1,1,0,0,0,1,0,0,1,0],[1,1,1,0,1,0,0,1,0,0],[1,1,1,1,0,1,0,0,0,0],[1,0,0,1,1,0,0,0,0,0],[1,0,1,0,0,1,0,0,0,0],[1,1,0,0,0,0,1,0,0,0],[0,0,0,1,0,0,0,0,0,1],[1,0,1,0,0,0,0,0,0,0],[0,0,1,0,0,1,0,0,0,0],[1,0,1,0,0,0,0,0,0,0],
[1,0,1,1,1,0,0,0,0,0],[0,0,0,0,0,1,0,1,0,0],[1,1,1,1,0,0,0,0,0,0],[1,0,1,0,0,0,0,0,0,0],[0,0,1,0,0,1,0,0,0,0],[1,1,0,1,0,0,0,0,0,1],[1,0,1,0,1,0,0,0,1,0],[0,0,0,1,1,0,0,0,0,0],[1,0,0,1,0,0,0,0,0,0],[1,0,0,0,1,1,0,0,0,0],
[1,0,0,1,0,0,0,0,0,0],])
 # Cada sublista es una toma y cada elemento de la sublista (indice 0,1,2..) es el actor, siendo 1 si participa en la toma y 0 si no participa en la toma

num_tomas, num_actores = actores_tomas.shape # Número de actores y tomas totales a realizar

print(f"Número de actores: {num_actores} actores\nNúmero de tomas: {num_tomas} tomas\nMáximo de tomas/día: {max_tomas} tomas/día")

max_iter = 500  # Número de iteraciones de búsqueda Tabú
tabu = 50  # Tamaño de la lista tabú

print(f"El número máximo de iteraciones es: {max_iter} y el tamaño de la lista Tabú: {tabu}")

Número de actores: 10 actores
Número de tomas: 30 tomas
Máximo de tomas/día: 6 tomas/día
El número máximo de iteraciones es: 500 y el tamaño de la lista Tabú: 50


In [3]:
# FUNCIÓN DE COSTE DEL RODAJE, suponiendo que cuesta un euro el dia que un actor va al set de rodaje, dando igual la cantidad de tomas (Los actores cobran lo mismo por cada día que van al set)
def calcular_coste(actores_tomas, plan_grabacion):
    coste = 0 # Se inicializa variable del coste total
    for dia in plan_grabacion: # Se recorre la lista de días de grabación
        grabar = set()
        for toma in dia: # Se recorre el número de toma que se hace por día
            for actor, graba in enumerate(actores_tomas[toma]): # Se recorren los actores por toma y se obtiene el actor (0-10) y si graba o no (0: NO, 1: sí)
              if graba == 1: # Si sí que graba, se incluye el actor en el set (no se repite al ser un set, por tanto, simplemente se muestra si en un dia un actor va al set)
                grabar.add(actor)
        coste += len(grabar) # Se suma al coste total de la grabación el coste por día de grabación

    return coste # Se devuelve el coste total de la grabación

In [4]:
# FUNCIÓN DE BÚSQUEDA TABÚ
def busqueda_tabu(actores_tomas, num_tomas, max_tomas, max_iter, tabu):
    # GENERACIÓN DE UNA SOLUCIÓN INICIAL ALEATORIA: Diversificación
    random.seed(42) # Se crea una semilla para que la solución inicial siempre sea la misma al ejecutar
    inicio = list(range(num_tomas)) # Se crea una lista con los números de tomas (0-30)
    random.shuffle(inicio) # Se desordenan los elementos
    plan_inicial = [inicio[i:i+max_tomas]for i in range(0, len(inicio), max_tomas)] # Se agrupan las tomas en grupos de 6 (el máximo por día)
    plan_actual = plan_inicial # El plan actual es el inicial generado, el cual se irá actualizando en las siguientes líneas
    coste = calcular_coste(actores_tomas, plan_actual) # Se calcula el coste con la configuración del plan de rodaje inicial

    # DECLARACIÓN DE LA LISTA TABÚ Y
    lista_tabu = [] # Se inicializa la lista Tabú

    # SE COMIENZA EL PROCESO DE ITERACIONES (max_iter)
    for x in range(max_iter): # Se crea el proceso x iteraciones máximas

        # BÚSQUEDA DE VECINDAD LOCAL: Se intercambian tomas entre días
        vecino = [] # Se inicializa la lista de vecinos
        for i in range(len(plan_inicial)): # Se recorren los días de la planificación
            for j in range(i + 1, len(plan_inicial)): # Se recorre la planificación de 1 a 30 (para que no se repitan los pares de días)
              # Se generan Ids de lista aleatorios, para crear nuevo planing intercambiando tomas entre dias con los Ids
              id1 = random.randint(0, len(plan_inicial[i]) - 1)
              id2 = random.randint(0, len(plan_inicial[j]) - 1)
              nuevo_plan = copy.deepcopy(plan_inicial) # Se crea una copia del plan inicial para no transcribirlo con los cambios a realizar
              nuevo_plan[i][id1], nuevo_plan[j][id2] = nuevo_plan[j][id2], nuevo_plan[i][id1] # Se intercambian las tomas entre los días, para generar nuevo planning
              vecino.append(nuevo_plan) # Se añade el plan a la lisat de vecinos

        validas = [i for i in vecino if i not in lista_tabu] # Se filtran los vecinos que no están en la lista tabú, siendo validas
        vecino_min = min(validas, key=lambda v: calcular_coste(actores_tomas, v), default=None) # Se elige el vecino con menor coste. Si la lista está vacia, se devuelve none

        # COMPROBAR EL COSTE DEL MEJOR VECINO
        if calcular_coste(actores_tomas, vecino_min) < coste:
            plan_actual = vecino_min # Si el coste es menor, el planning actual será este
            coste = calcular_coste(actores_tomas, vecino_min) # Se calcula de nuevo el coste

        # AÑADIR EL VECINO MÍNIMO A LA LISTA TABÚ
        if vecino_min:
            lista_tabu.append(vecino_min) # Añadir el vecino con mínimo coste a la lista tabú
            if len(lista_tabu) > tabu: # Se elimina la solución más antigua si la lista supera el tamaño que se ha definido al principio
                lista_tabu.pop(0)

    return plan_actual, coste # Se devuelve el planing final y el coste

In [5]:
# PRUEBA DEL CÓDIGO DE BÚSQUEDA TABÚ
planning, coste = busqueda_tabu(actores_tomas, num_tomas, max_tomas, max_iter, tabu) # Se llama a la función

print("     --- Planificación del rodaje ---")
for i, dia in enumerate(planning, start=1): # Se recorre el planning final. El start = 1 es para empezar los dias en 1 y no en 0.
  nuevo_dia = [j+1 for j in dia] # Se suma 1 a cada toma, para que empiece en 1
  print(f"Día {i} de rodaje: TOMAS {nuevo_dia}")
print(f"Coste del rodaje: {coste} euros")

     --- Planificación del rodaje ---
Día 1 de rodaje: TOMAS [20, 5, 11, 27, 23, 7]
Día 2 de rodaje: TOMAS [6, 13, 12, 16, 10, 26]
Día 3 de rodaje: TOMAS [30, 22, 17, 28, 2, 14]
Día 4 de rodaje: TOMAS [19, 3, 18, 29, 15, 25]
Día 5 de rodaje: TOMAS [8, 9, 24, 1, 4, 21]
Coste del rodaje: 36 euros
